# Setup

In [5]:
import numpy as np
import seaborn as sns
import csv

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func
from ipywidgets import interact

from flask import Flask, jsonify


In [6]:
# create and save database
execfile("Setup and Organization/Database Setup.py")
# database setup
engine = create_engine("sqlite:///Resources/database.sqlite")
conn=engine.connect()


/var/folders/vd/r058js812ld0xgwzbqh1j9c00000gn/T/ipykernel_53221/525720125.py:10: SADeprecationWarning: The AutomapBase.prepare.reflect parameter is deprecated and will be removed in a future release.  Reflection is enabled when AutomapBase.prepare.autoload_with is passed.
  base.prepare(engine,reflect=True)


# Explore Database

In [11]:
# explore database
inspector = inspect(engine)
inspector.get_table_names()

['activity',
 'bmi',
 'gender',
 'health',
 'occupation',
 'person',
 'sleep',
 'sleep_disorder']

In [14]:
# Get a list of column names and types
columns = inspector.get_columns('person')
for c in columns:
    print(c['name'], c["type"])
columns

index BIGINT
id BIGINT
gender_id BIGINT
age BIGINT
occupation_id BIGINT


[{'name': 'index',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'id',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'gender_id',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'age',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'primary_key': 0},
 {'name': 'occupation_id',
  'type': BIGINT(),
  'nullable': True,
  'default': None,
  'primary_key': 0}]

# Reflect and query data

In [15]:
# reflect an existing database into a new model
base= automap_base()
# reflect the tables
base.prepare(engine,reflect=True)


/var/folders/vd/r058js812ld0xgwzbqh1j9c00000gn/T/ipykernel_53221/90311862.py:4: SADeprecationWarning: The AutomapBase.prepare.reflect parameter is deprecated and will be removed in a future release.  Reflection is enabled when AutomapBase.prepare.autoload_with is passed.
  base.prepare(engine,reflect=True)


In [16]:
session= Session(engine)

In [19]:
engine.excecute(text('SELECT * FROM person LIMIT 5')).fetchall()

AttributeError: 'Engine' object has no attribute 'excecute'

In [18]:
# get age from person.csv    X
person = Base.person
# get sleep duration from sleep.csv    Y
# get quality of sleep from sleep.csv    marker size
# get stress_level from health.csv    marker color
# get age and id from person.csv     text value
# toggle between gender


AttributeError: type object 'Base' has no attribute 'person'